xgboost total points by Pierre

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


ModuleNotFoundError: No module named 'pandas'

In [ ]:
data = pd.read_csv("cleaned_merged_seasons.csv")

In [ ]:
# valeurs manquantes
data = data.dropna(subset=['team_x'])

# Normaliser les caractéristiques
features_to_normalize = ["creativity", "ict_index", "influence", "threat"]
scaler = MinMaxScaler()
data[features_to_normalize] = scaler.fit_transform(data[features_to_normalize])

# Encode
data_encoded = pd.get_dummies(data, columns=["season_x", "position", "team_x", "kickoff_time", "opp_team_name"], drop_first=True)

# data type int
data_encoded['was_home'] = data_encoded['was_home'].astype(int)

# Retirer la colonne "name"
player_names = data_encoded['name'].reset_index(drop=True)
data_encoded = data_encoded.drop('name', axis=1)

In [ ]:
print(player_names)

In [ ]:
# Séparation des données en entrée (X) et sortie (y)
X = data_encoded.drop('total_points', axis=1)
y = data_encoded['total_points']

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test, player_names_train, player_names_test = train_test_split(
    X, y, player_names, test_size=0.2, random_state=42
)

# Redimensionnement des données pour être utilisées dans un modèle LSTM
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))


In [ ]:
# Création du modèle LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
player_names_test = player_names_test.reset_index(drop=True)

results_df = pd.DataFrame({'Name': player_names_test, 'Predicted_Total_Points': predictions})

print(results_df)

In [ ]:
# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")